In [9]:
import requests
import json
import pandas as pd

In [10]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://shopee.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

In [11]:
URL = "https://api.tiki.vn/raiden/v2/menu-config?platform=desktop"

response = requests.get(URL, headers=HEADERS)

if response.status_code == 200:
    data = response.json()
    print("API Response:", json.dumps(data, indent=4))
else:
    print("Failed to fetch data:", response.status_code)

API Response: {
    "menu_block": {
        "title": "Danh m\u1ee5c",
        "items": [
            {
                "text": "Nh\u00e0 S\u00e1ch Tiki",
                "icon_url": "https://salt.tikicdn.com/ts/category/ed/20/60/afa9b3b474bf7ad70f10dd6443211d5f.png",
                "link": "https://tiki.vn/nha-sach-tiki/c8322"
            },
            {
                "text": "Nh\u00e0 C\u1eeda - \u0110\u1eddi S\u1ed1ng",
                "icon_url": "https://salt.tikicdn.com/ts/category/f6/22/46/7e2185d2cf1bca72d5aeac385a865b2b.png",
                "link": "https://tiki.vn/nha-cua-doi-song/c1883"
            },
            {
                "text": "\u0110i\u1ec7n Tho\u1ea1i - M\u00e1y T\u00ednh B\u1ea3ng",
                "icon_url": "https://salt.tikicdn.com/ts/category/54/c0/ff/fe98a4afa2d3e5142dc8096addc4e40b.png",
                "link": "https://tiki.vn/dien-thoai-may-tinh-bang/c1789"
            },
            {
                "text": "\u0110\u1ed3 Ch\u01a1i - M\u1eb9 & B\

### GET GROUP ID

In [12]:
# Extract group data from API respone
group = data["menu_block"]["items"]
group_list = []

for category in group:
    text = category["text"]
    link = category["link"]
    # Extract group ID from the link
    group_id = link.split("/")[-1][1:]  
    group_list.append([group_id, text])

In [13]:
# Convert to DataFrame
group = pd.DataFrame(group_list, columns=["Group_ID", "Name"])
group

,Group_ID,Name
0,8322,Nhà Sách Tiki
1,1883,Nhà Cửa - Đời Sống
2,1789,Điện Thoại - Máy Tính Bảng
3,2549,Đồ Chơi - Mẹ & Bé
4,1815,Thiết Bị Số - Phụ Kiện Số
5,1882,Điện Gia Dụng
6,1520,Làm Đẹp - Sức Khỏe
7,8594,Ô Tô - Xe Máy - Xe Đạp
8,931,Thời trang nữ
9,4384,Bách Hóa Online


### GET PARENT CATEGORY ID

In [14]:
parent_category_list = []

for group_id in group["Group_ID"]:
    URL = f"https://tiki.vn/api/v2/categories?parent_id={group_id}"   
    response = requests.get(URL, headers=HEADERS)
    
    if response.status_code == 200:
         data = response.json()
         for parent_category in data["data"]:
             id = parent_category["id"]
             name = parent_category["name"]
             
             # Get group name 
             group_name = group[group["Group_ID"] == group_id]["Name"].item()
             
             parent_category_list.append([id, name, group_id, group_name])

In [15]:
# Convert to DataFrame
parent_category = pd.DataFrame(parent_category_list, columns=["Parent_ID", "Parent_Name", "Group_ID", "Group_Name"])
parent_category

,Parent_ID,Parent_Name,Group_ID,Group_Name
0,320,English Books,8322,Nhà Sách Tiki
1,316,Sách tiếng Việt,8322,Nhà Sách Tiki
2,7741,Văn phòng phẩm,8322,Nhà Sách Tiki
3,18328,Quà lưu niệm,8322,Nhà Sách Tiki
4,1951,Dụng cụ nhà bếp,1883,Nhà Cửa - Đời Sống
...,...,...,...,...
203,8352,Phụ kiện túi,976,Túi thời trang nữ
204,5337,Túi xách công sở nam,27616,Túi thời trang nam
205,958,Túi đeo chéo nam,27616,Túi thời trang nam
206,49650,"Túi bao tử, túi đeo bụng",27616,Túi thời trang nam


In [16]:
# Store to test (encoding utf-8-sig to avoid BOM)
parent_category.to_csv("tiki_parent_category_automatic.csv", index=False, encoding="utf-8-sig")